In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from scipy.special import erfc, erfcinv
import os, glob, pickle
import json
import matplotlib.pyplot as plt
import libstempo as t2
from enterprise.signals import utils as ent_utils
from enterprise import constants as const
from dr3_noise.models import model_singlepsr_noise
from enterprise_extensions.blocks import common_red_noise_block
from enterprise.signals import signal_base
from h5pulsar import Pulsar, FilePulsar
#from enterprise.pulsar import Pulsar
import IPTA_DR2_analysis.utils as dr2u
from IPTA_DR2_analysis.model_blocks import adv_noise_block

In [ ]:
from la_forge import core as co
import defiant
from defiant import OptimalStatistic
from defiant import utils, orf_functions
from defiant import plotting as defplot
from defiant.extra import mdc1_utils
import tqdm
from IPython.display import display, Math

In [ ]:
%matplotlib inline
%config IPython.matplotlib.backend = "retina"
from matplotlib import rcParams
rcParams["savefig.dpi"] = 300
rcParams["figure.dpi"] = 300

Here we would like to generate MCOS results using our new runs. We'll use DEFIANT to do this. We'll use the following settings throughout:
- Multiple ORF
- Noise marginalization
- Uncertainty sampling
- Power law model (fixed gamma)

In order to get these results, we need to load the following:

## Load PSRs, PTA object, CRN chain

In [ ]:
All_MCOS = {}

In [ ]:
# UPDATE THIS
ds = 'lite_unfiltered_53'
if ds == 'lite' or ds == 'litec_filtered':
    Nfreqs = 7
else:
    Nfreqs = 13
project_path = '/vast/palmer/home.grace/bbl29/IPTA_DR2_analysis'
dataset = f'{project_path}/data/{ds}_ePSRs'
if 'lite_unfiltered' in ds:
    noisedict_path = f'{project_path}/noisedicts/dr2lite_unfiltered_advnoise.json'
elif not ds == 'lite':
    noisedict_path = f'{project_path}/noisedicts/dr2full_advnoise.json'
else:
    noisedict_path = f'{project_path}/noisedicts/dr2{ds}_advnoise.json'
if ds == 'edr2':
    coredir = f'/vast/palmer/home.grace/bbl29/project/IPTA_DR2_analysis/edr2'
    figsave = f'{project_path}/figs/edr2/CRN{Nfreqs}_g4p3_advnoise'
else:
    coredir = f'/vast/palmer/home.grace/bbl29/project/IPTA_DR2_analysis/dr2{ds}'
    figsave = f'{project_path}/figs/dr2{ds}/CRN{Nfreqs}_g4p3_advnoise'

In [ ]:
Nfreqs, noisedict_path

In [ ]:
psrs = []
psr_paths = np.sort(glob.glob(f'{dataset}/J*'))
try:
    for psr_path in psr_paths:
        with open(psr_path,'rb') as f:
            psrs.append(FilePulsar(f))
except:
    for psr_path in psr_paths:
        with open(psr_path,'rb') as f:
            psrs.append(pickle.load(f))
len(psrs), psr_paths[0]

We have a fixed gamma run and a varied gamma run. Let's make both ptas and load both runs.

The pta objects are setup to be exactly the same as what was used for the MCMC run.

In [ ]:
# load noise params
with open(noisedict_path,'r') as f:
    noise_params = json.load(f)

# set up model
max_mjd = np.max([np.max(psr.toas) for psr in psrs])
min_mjd = np.min([np.min(psr.toas) for psr in psrs])
Tspan = max_mjd - min_mjd
crn = common_red_noise_block(psd='powerlaw', prior='log-uniform', Tspan=Tspan,
                             components=Nfreqs, gamma_val=13/3,
                             logmin=-18, logmax=-12,
                             orf=None, name='crn')

# loop through pulsars, add noise
models = []
for psr in tqdm.tqdm(psrs):
    if ds in ['litec', 'edr2'] or 'unfiltered' in ds:
        noise = adv_noise_block(psr, full_pta_analysis=True, dataset='full', psr_model=True,
                                tm_marg=True, tm_svd=True)
    else:
        noise = adv_noise_block(psr, full_pta_analysis=True, dataset=ds, psr_model=True,
                                tm_marg=True, tm_svd=True)
    signals = crn + noise
    models.append(signals(psr))
pta = signal_base.PTA(models)
pta.set_default_params(noise_params)

In [ ]:
corepath = f'{coredir}/CRN{Nfreqs}_g4p3_advnoise/core.h5'
core = co.Core(corepath=corepath, burn=0)

## Get MCOS distributions

We will first compare the full distributions with uncertainty sampling, then we will compare the SNR distributions.

In [ ]:
MCOS_obj = OptimalStatistic(psrs, pta=pta, gwb_name='crn', core=core, orfs=['hd','mp','dp'])

In [ ]:
# Fixed gamma results: HD, DP, MP. Get extra samples since this is what's used in the paper
A2,A2s,idx = MCOS_obj.compute_OS(N=3000, return_pair_vals=False, pair_covariance=False, gamma=13/3)
MCOS_results = {'A2_mean':A2, 'idx':idx}
MCOS_results['A2_sigma'] = np.array([np.diag(np.sqrt(np.abs(A2s[i]))) for i in range(len(A2s))])

In [ ]:
# compute additional statistics: A2, SNR, Bayesian p-value, Bayesian SNR
MCOS_results['A2'] = utils.uncertainty_sample(A2,A2s,pfos=False,mcos=True)
MCOS_results['SNR'] = MCOS_results['A2_mean']/MCOS_results['A2_sigma']
MCOS_results['pB'] = 1/2*erfc(MCOS_results['SNR']/np.sqrt(2))
MCOS_results['pB_mean'] = np.mean(MCOS_results['pB'], axis=0)
MCOS_results['BSNR'] = np.sqrt(2)*erfcinv(2*MCOS_results['pB_mean'])

In [ ]:
# save SNRs for dataset comparison
All_MCOS[ds] = {}
All_MCOS[ds]['SNR'] = MCOS_results['SNR']
All_MCOS[ds]['BSNR'] = MCOS_results['BSNR']
All_MCOS[ds]['A2'] = MCOS_results['A2']

In [ ]:
All_MCOS

In [ ]:
NMOS_savepath = f'/vast/palmer/home.grace/bbl29/project/IPTA_DR2_analysis/NMOS.pkl'
with open(NMOS_savepath, 'wb') as f:
    pickle.dump(All_MCOS, f)

Can also just load the pickle directly

In [ ]:
coredir

In [ ]:
NMOS_savepath = f'/vast/palmer/home.grace/bbl29/project/IPTA_DR2_analysis/NMOS.pkl'
with open(NMOS_savepath, 'rb') as f:
    All_MCOS = pickle.load(f)

### Amplitudes

For uncertainty sampling, we are assuming the OS is a Gaussian

In [ ]:
figsave

In [ ]:
fig, ax = plt.subplots(figsize=(6,3))
plt.hist(MCOS_results['A2'][:,1],bins='auto',histtype='step',label=r'Monopole',density=True,range=[0,2e-29])
plt.hist(MCOS_results['A2'][:,0],bins='auto',histtype='step',label=r'HD',density=True,range=[0,2e-29])
plt.hist(MCOS_results['A2'][:,2],bins='auto',histtype='step',label=r'Dipole',density=True,range=[0,2e-29])
plt.hist((10**core('crn_log10_A'))**2,bins='auto',histtype='step',color='k',alpha=0.5,ls='--',
         label='Bayesian uncorrelated process',density=True,range=[0,2e-29])
plt.xlim([0,2e-29])
plt.title(fr'IPTA DR2{ds} | $A^2$ full distributions (fixed $\gamma = 13/3$)')
plt.xlabel('$A^2$')
plt.ylabel('$p(A^2)$')
plt.legend()
plt.savefig(f'{figsave}/OS.png', dpi=300, bbox_inches='tight')
plt.show()

### SNRs

Note here we are assuming the null-distribution is a Gaussian, which is wrong

In [ ]:
fig, ax = plt.subplots(figsize=(6,3))
labels = ['HD', 'Monopole', 'Dipole']
colors = ['C1', 'C0', 'C2']
for i in range(3):
    plt.hist(MCOS_results['SNR'][:,i],bins='auto',histtype='step',
             density=True,color=colors[i],label=labels[i]+f' | BSNR = {MCOS_results["BSNR"][i]:0.2f}')
    ax.axvline(MCOS_results['BSNR'][i], color=colors[i])
#plt.xlim([0,2e-29])
plt.title(fr'IPTA DR2{ds} | S/N distributions (fixed $\gamma = 13/3$)')
plt.xlabel('S/N')
plt.ylabel('$p(S/N)$')
plt.legend()
plt.savefig(f'{figsave}/SNR.png', dpi=300, bbox_inches='tight')
plt.show()

### p-values

Note here we are assuming the null-distribution is a Gaussian, which is wrong

In [ ]:
fig, ax = plt.subplots(figsize=(6,3))
labels = ['HD', 'Monopole', 'Dipole']
colors = ['C1', 'C0', 'C2']
for i in range(3):
    plt.hist(np.log10(MCOS_results['pB'][:,i]),bins=40,histtype='step',
             range=[np.max([-6,np.min(np.log10(MCOS_results['pB'][:,i]))]),0],
             density=True,color=colors[i],label=labels[i]+fr' | $p_B$ = {MCOS_results["pB_mean"][i]:0.2f}')
    ax.axvline(np.log10(MCOS_results['pB_mean'][i]), color=colors[i])
#plt.xlim([0,2e-29])
plt.title(fr'IPTA DR2{ds} | $p_B$ distributions (fixed $\gamma = 13/3$)')
plt.xlabel(r'$\log_{10}p_B$')
plt.ylabel('$p(p_B)$')
plt.xlim([-6,0])
plt.legend()
plt.savefig(f'{figsave}/p_values.png', dpi=300, bbox_inches='tight')
plt.show()

# Paper versions

### Compare datasets -- Old version (Lite filtered 21, Combined 21, Full 21)

In [ ]:
fig, ax = plt.subplots(figsize=(6,3))
datasets = ['full', 'lite_unfiltered_53', 'edr2']
labels = ['Full DR2', 'DR2 Lite', 'EDR2']
#datasets = ['litec', 'litec_filtered']
#labels = ['EDR2', 'EDR2_filtered']
for i in range(len(datasets)):
    plt.hist(All_MCOS[datasets[i]]['SNR'][:,0],bins='auto',histtype='step',
             density=True,color=f'C{i}',label=labels[i])#+f' | BSNR = {All_MCOS[datasets[i]]["BSNR"][0]:0.2f}')
    ax.axvline(All_MCOS[datasets[i]]['BSNR'][0], color=f'C{i}', lw=1, ls='--')
#plt.xlim([0,2e-29])
#plt.title(fr'IPTA DR2{ds} | S/N distributions (fixed $\gamma = 13/3$)')
plt.xlabel('HD S/N', fontsize='x-large')
plt.ylabel('PDF', fontsize='x-large')
plt.legend(frameon=False)
plt.savefig(f'{project_path}/figs/dataset_comparisons/gwb/OS_SNR.png',
            dpi=300, bbox_inches='tight')
plt.show()

Let's have a version with all 3 ORFs

In [ ]:
print(All_MCOS['full'])

In [ ]:
#fig, ax = plt.subplots(figsize=(6,3))
datasets = ['full', 'lite_unfiltered_53', 'edr2']
labels = ['Full DR2', 'DR2 Lite', 'EDR2']
#datasets = ['litec', 'litec_filtered']
#labels = ['EDR2', 'EDR2_filtered']
orf_labels = ['Hellings-Downs', 'Monopole', 'Dipole']
fig, ax = plt.subplots(3,1,figsize=(6,6),sharex=True)
for j in range(len(orf_labels)):
    for i in range(len(datasets)):
        ax[j].hist(All_MCOS[datasets[i]]['SNR'][:,j],bins='auto',histtype='step',
                   density=True,color=f'C{i}',
                   label=labels[i])#+f' | BSNR = {All_MCOS[datasets[i]]["BSNR"][0]:0.2f}')
        ax[j].axvline(All_MCOS[datasets[i]]['BSNR'][j], color=f'C{i}', lw=1, ls='--')
    ax[j].set_ylabel(f'PDF', fontsize='x-large')
#ax[0].legend(frameon=False)
for j in range(len(orf_labels)):
    xlim = ax[j].get_xlim()
    ylim = ax[j].get_ylim()
    ax[j].text(xlim[1]-np.diff(xlim)/2.8, ylim[1]*0.8, orf_labels[j], fontsize='large', weight='bold')
    ax[j].grid(lw=0.3)
    ax[j].set_yticks([])
#plt.xlim([0,2e-29])
#plt.title(fr'IPTA DR2{ds} | S/N distributions (fixed $\gamma = 13/3$)')
ax[-1].set_xlabel('S/N', fontsize='x-large')
fig.subplots_adjust(hspace=0)
plt.savefig(f'{project_path}/figs/dataset_comparisons/gwb/MCOS_SNR.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#fig, ax = plt.subplots(figsize=(6,3))
datasets = ['full', 'lite_unfiltered_53', 'edr2']
labels = ['Full DR2', 'DR2 Lite', 'EDR2']
#datasets = ['litec', 'litec_filtered']
#labels = ['EDR2', 'EDR2_filtered']
orf_labels = ['Hellings-Downs', 'Monopole', 'Dipole']
fig, ax = plt.subplots(3,1,figsize=(6,6),sharex=True)
for j in range(len(orf_labels)):
    for i in range(len(datasets)):
        ax[j].hist(All_MCOS[datasets[i]]['SNR'][:,j]**2,bins='auto',histtype='step',
                   density=True,color=f'C{i}',
                   label=labels[i])#+f' | BSNR = {All_MCOS[datasets[i]]["BSNR"][0]:0.2f}')
        ax[j].axvline(All_MCOS[datasets[i]]['BSNR'][j]**2, color=f'C{i}', lw=1, ls='--')
    ax[j].set_ylabel(f'PDF', fontsize='x-large')
#ax[0].legend(frameon=False)
ax[0].set_xlim([0,10])
for j in range(len(orf_labels)):
    xlim = ax[j].get_xlim()
    ylim = ax[j].get_ylim()
    ax[j].text(xlim[1]-4, ylim[1]*0.8, orf_labels[j], fontsize='large', weight='bold')
    ax[j].grid(lw=0.3)
#plt.xlim([0,2e-29])
#plt.title(fr'IPTA DR2{ds} | S/N distributions (fixed $\gamma = 13/3$)')
ax[-1].set_xlabel(r'S/N$^2$', fontsize='x-large')
fig.subplots_adjust(hspace=0)
plt.savefig(f'{project_path}/figs/dataset_comparisons/gwb/MCOS_SNR_sq.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#fig, ax = plt.subplots(figsize=(6,3))
datasets = ['full', 'lite_unfiltered_53', 'edr2']
labels = ['Full DR2', 'DR2 Lite', 'EDR2']
#datasets = ['litec', 'litec_filtered']
#labels = ['EDR2', 'EDR2_filtered']
orf_labels = ['Hellings-Downs', 'Monopole', 'Dipole']
fig, ax = plt.subplots(3,1,figsize=(6,6),sharex=True)
for j in range(len(orf_labels)):
    for i in range(len(datasets)):
        mask = All_MCOS[datasets[i]]['A2'][:,j] > 0
        ax[j].hist(All_MCOS[datasets[i]]['A2'][:,j][mask],bins='auto',histtype='step',
                   density=True,color=f'C{i}',
                   label=labels[i])#+f' | BSNR = {All_MCOS[datasets[i]]["BSNR"][0]:0.2f}')
        #ax[j].axvline(All_MCOS[datasets[i]]['A2'][j], color=f'C{i}', lw=1, ls='--')
    ax[j].set_ylabel(f'PDF', fontsize='x-large')
#ax[0].legend(frameon=False)
ax[0].set_xlim([0,3e-29])
for j in range(len(orf_labels)):
    xlim = ax[j].get_xlim()
    ylim = ax[j].get_ylim()
    ax[j].text(xlim[1]-np.diff(xlim)/2.8, ylim[1]*0.8, orf_labels[j], fontsize='large', weight='bold')
    ax[j].grid(lw=0.3)
    ax[j].set_yticks([])
plt.xlim([0,3e-29])
#plt.title(fr'IPTA DR2{ds} | S/N distributions (fixed $\gamma = 13/3$)')
ax[-1].set_xlabel(r'$\hat{A}^2$', fontsize='x-large')
fig.subplots_adjust(hspace=0)
plt.savefig(f'{project_path}/figs/dataset_comparisons/gwb/MCOS_Asq.png', dpi=300, bbox_inches='tight')
plt.show()

# Pair-covariant MCOS

In [ ]:
All_PCMCOS = {}

In [ ]:
PCOS_obj = OptimalStatistic(psrs, pta=pta, gwb_name='crn', core=core, orfs=['hd','mp','dp'])
PCOS_obj.set_orf(['hd','dipole','monopole'], pcmc_orf='hd')

In [ ]:
A2,A2s,idx = PCOS_obj.compute_OS(N=100, pair_covariance=True, return_pair_vals=False, gamma=13/3)
PCOS_results = {'A2_mean':A2, 'idx':idx}
PCOS_results['A2_sigma'] = np.array([np.diag(np.sqrt(np.abs(A2s[i]))) for i in range(len(A2s))])

In [ ]:
# compute additional statistics: A2, SNR, Bayesian p-value, Bayesian SNR
PCOS_results['A2'] = utils.uncertainty_sample(A2,A2s,pfos=False,mcos=True)
PCOS_results['SNR'] = PCOS_results['A2_mean']/PCOS_results['A2_sigma']
PCOS_results['pB'] = 1/2*erfc(PCOS_results['SNR']/np.sqrt(2))
PCOS_results['pB_mean'] = np.mean(PCOS_results['pB'], axis=0)
PCOS_results['BSNR'] = np.sqrt(2)*erfcinv(2*PCOS_results['pB_mean'])

In [ ]:
# save SNRs for dataset comparison
All_PCOS[ds] = {}
All_PCOS[ds]['A2'] = PCOS_results['A2']
All_PCOS[ds]['SNR'] = PCOS_results['SNR']
All_PCOS[ds]['BSNR'] = PCOS_results['BSNR']

In [ ]:
PCMCOS_savepath = f'/vast/palmer/home.grace/bbl29/project/IPTA_DR2_analysis/PCMCOS.pkl'
with open(PCMCOS_savepath, 'wb') as f:
    pickle.dump(All_PCMCOS, f)

Can also just load the pickle directly

In [ ]:
PCMCOS_savepath = f'/vast/palmer/home.grace/bbl29/project/IPTA_DR2_analysis/PCMCOS.pkl'
with open(PCMCOS_savepath, 'rb') as f:
    All_PCMCOS = pickle.load(f)

In [ ]:
All_PCOS

In [ ]:
fig, ax = plt.subplots(figsize=(6,3))
plt.hist(PCOS_results['A2'][:,1],bins='auto',histtype='step',label=r'Monopole',density=True,range=[0,3e-29])
plt.hist(PCOS_results['A2'][:,0],bins='auto',histtype='step',label=r'HD',density=True,range=[0,3e-29])
plt.hist(PCOS_results['A2'][:,2],bins='auto',histtype='step',label=r'Dipole',density=True,range=[0,3e-29])
plt.hist((10**core('crn_log10_A'))**2,bins='auto',histtype='step',color='k',alpha=0.5,ls='--',
         label='Bayesian uncorrelated process',density=True,range=[0,3e-29])
plt.xlim([0,3e-29])
plt.title(fr'IPTA DR2{ds} | $A^2$ full distributions (fixed $\gamma = 13/3$)')
plt.xlabel('$A^2$')
plt.ylabel('$p(A^2)$')
plt.legend()
plt.savefig(f'{figsave}/PCOS.png', dpi=300, bbox_inches='tight')
plt.show()

### SNRs

Note here we are assuming the null-distribution is a Gaussian, which is wrong

In [ ]:
fig, ax = plt.subplots(figsize=(6,3))
labels = ['HD', 'Monopole', 'Dipole']
colors = ['C1', 'C0', 'C2']
for i in range(3):
    plt.hist(PCOS_results['SNR'][:,i],bins='auto',histtype='step',
             density=True,color=colors[i],label=labels[i]+f' | BSNR = {PCOS_results["BSNR"][i]:0.2f}')
    ax.axvline(PCOS_results['BSNR'][i], color=colors[i])
#plt.xlim([0,2e-29])
plt.title(fr'IPTA DR2{ds} | S/N distributions (fixed $\gamma = 13/3$)')
plt.xlabel('S/N')
plt.ylabel('$p(S/N)$')
plt.legend()
plt.savefig(f'{figsave}/PC_SNR.png', dpi=300, bbox_inches='tight')
plt.show()

### p-values

Note here we are assuming the null-distribution is a Gaussian, which is wrong

In [ ]:
fig, ax = plt.subplots(figsize=(6,3))
labels = ['HD', 'Monopole', 'Dipole']
colors = ['C1', 'C0', 'C2']
for i in range(3):
    plt.hist(np.log10(PCMCOS_results['pB'][:,i]),bins=40,histtype='step',
             range=[np.max([-6,np.min(np.log10(PCMCOS_results['pB'][:,i]))]),0],
             density=True,color=colors[i],label=labels[i]+fr' | $p_B$ = {PCMCOS_results["pB_mean"][i]:0.2f}')
    ax.axvline(np.log10(PCMCOS_results['pB_mean'][i]), color=colors[i])
#plt.xlim([0,2e-29])
plt.title(fr'IPTA DR2{ds} | $p_B$ distributions (fixed $\gamma = 13/3$)')
plt.xlabel(r'$\log_{10}p_B$')
plt.ylabel('$p(p_B)$')
plt.xlim([-6,0])
plt.legend()
plt.savefig(f'{figsave}/PC_p_values.png', dpi=300, bbox_inches='tight')
plt.show()

### Compare datasets -- New paper version (Lite unfiltered 21, Combined 21, Full 53)

In [ ]:
#fig, ax = plt.subplots(figsize=(6,3))
datasets = ['full', 'lite_unfiltered_21', 'litec']
labels = ['Full DR2', 'DR2 Lite', 'EDR2']
#datasets = ['litec', 'litec_filtered']
#labels = ['EDR2', 'EDR2_filtered']
orf_labels = ['HD', 'Monopole', 'Dipole']
fig, ax = plt.subplots(3,1,figsize=(6,6),sharex=True)
for j in range(len(orf_labels)):
    for i in range(len(datasets)):
        ax[j].hist(All_MCOS[datasets[i]]['SNR'][:,j],bins='auto',histtype='step',
                   density=True,color=f'C{i}',
                   label=labels[i])#+f' | BSNR = {All_MCOS[datasets[i]]["BSNR"][0]:0.2f}')
        ax[j].axvline(All_MCOS[datasets[i]]['BSNR'][j], color=f'C{i}', lw=1, ls='--')
    ax[j].set_ylabel(f'PDF', fontsize='x-large')
#ax[0].legend(frameon=False)
for j in range(len(orf_labels)):
    xlim = ax[j].get_xlim()
    ylim = ax[j].get_ylim()
    ax[j].text(xlim[0]+1, ylim[1]*0.8, orf_labels[j], fontsize='large', weight='bold')
    ax[j].grid(lw=0.3)
#plt.xlim([0,2e-29])
#plt.title(fr'IPTA DR2{ds} | S/N distributions (fixed $\gamma = 13/3$)')
ax[-1].set_xlabel('S/N', fontsize='x-large')
plt.savefig(f'{project_path}/figs/dataset_comparisons/gwb/MCOS_SNR_unfiltered.png',
            dpi=300, bbox_inches='tight')
fig.subplots_adjust(hspace=0)
plt.show()

### Compare datasets -- New paper version (Lite unfiltered 43, Combined 21, Full 53)

In [ ]:
#fig, ax = plt.subplots(figsize=(6,3))
datasets = ['full', 'lite_unfiltered_43', 'litec']
labels = ['Full DR2', 'DR2 Lite', 'EDR2']
#datasets = ['litec', 'litec_filtered']
#labels = ['EDR2', 'EDR2_filtered']
orf_labels = ['HD', 'Monopole', 'Dipole']
fig, ax = plt.subplots(3,1,figsize=(6,6),sharex=True)
for j in range(len(orf_labels)):
    for i in range(len(datasets)):
        ax[j].hist(All_MCOS[datasets[i]]['SNR'][:,j],bins='auto',histtype='step',
                   density=True,color=f'C{i}',
                   label=labels[i])#+f' | BSNR = {All_MCOS[datasets[i]]["BSNR"][0]:0.2f}')
        ax[j].axvline(All_MCOS[datasets[i]]['BSNR'][j], color=f'C{i}', lw=1, ls='--')
    ax[j].set_ylabel(f'PDF', fontsize='x-large')
#ax[0].legend(frameon=False)
for j in range(len(orf_labels)):
    xlim = ax[j].get_xlim()
    ylim = ax[j].get_ylim()
    ax[j].text(xlim[0]+1, ylim[1]*0.8, orf_labels[j], fontsize='large', weight='bold')
    ax[j].grid(lw=0.3)
#plt.xlim([0,2e-29])
#plt.title(fr'IPTA DR2{ds} | S/N distributions (fixed $\gamma = 13/3$)')
ax[-1].set_xlabel('S/N', fontsize='x-large')
plt.savefig(f'{project_path}/figs/dataset_comparisons/gwb/MCOS_SNR_unfiltered_43.png',
            dpi=300, bbox_inches='tight')
fig.subplots_adjust(hspace=0)
plt.show()

### Compare datasets -- Presentation version 21 psrs each (Combined unfiltered, Lite unfiltered, Lite filtered, Combined filtered)

In [ ]:
#fig, ax = plt.subplots(figsize=(6,3))
datasets = ['litec', 'lite', 'lite_unfiltered_21', 'litec_filtered']
labels = ['EDR2', 'DR2 Lite filtered', 'DR2 Lite', 'EDR2 filtered']
#datasets = ['litec', 'litec_filtered']
#labels = ['EDR2', 'EDR2_filtered']
orf_labels = ['HD', 'Monopole', 'Dipole']
fig, ax = plt.subplots(3,1,figsize=(6,6),sharex=True)
for j in range(len(orf_labels)):
    for i in range(len(datasets)):
        ax[j].hist(All_MCOS[datasets[i]]['SNR'][:,j],bins='auto',histtype='step',
                   density=True,color=f'C{i}',
                   label=labels[i])#+f' | BSNR = {All_MCOS[datasets[i]]["BSNR"][0]:0.2f}')
        ax[j].axvline(All_MCOS[datasets[i]]['BSNR'][j], color=f'C{i}', lw=1, ls='--')
    ax[j].set_ylabel(f'PDF', fontsize='x-large')
#ax[0].legend(frameon=False)
for j in range(len(orf_labels)):
    xlim = ax[j].get_xlim()
    ylim = ax[j].get_ylim()
    ax[j].text(xlim[0]+1, ylim[1]*0.8, orf_labels[j], fontsize='large', weight='bold')
    ax[j].grid(lw=0.3)
#plt.xlim([0,2e-29])
#plt.title(fr'IPTA DR2{ds} | S/N distributions (fixed $\gamma = 13/3$)')
ax[-1].set_xlabel('S/N', fontsize='x-large')
plt.savefig(f'{project_path}/figs/dataset_comparisons/gwb/MCOS_SNR_compare_filters.png',
            dpi=300, bbox_inches='tight')
fig.subplots_adjust(hspace=0)
plt.show()

## MCOS - EDR2 filtered dataset

In [ ]:
figsave

# PC test

Let's first do a test to see if there is any difference whether we use pair covariance or not.

Now make the OS

In [ ]:
MCOS_obj = OptimalStatistic(psrs, pta=pta, gwb_name='crn', core=core, orfs=['hd','mp','dp'])

### Generate OS results

In [ ]:
# Fixed gamma results: HD, DP, MP. Get extra samples since this is what's used in the paper
A2,A2s,idx = MCOS_obj.compute_OS(N=500, return_pair_vals=False, pair_covariance=False, gamma=13/3)
MCOS_results = {'A2':A2, 'A2_sigma':A2s, 'idx':idx}

In [ ]:
# Fixed gamma results: HD, DP, MP. Get extra samples since this is what's used in the paper
A2,A2s,idx = MCOS_obj.compute_OS(N=500, return_pair_vals=False, pair_covariance=True, gamma=13/3)
MCOS_results_pc = {'A2':A2, 'A2_sigma':A2s, 'idx':idx}

It turns out the PCMCOS is actually broken right now, so that's why we are not using it.